# Librerias

In [1]:
## Se importan librerias para leer uno de los archivos Json provistos y hacer el filtro de la categoria 
## que vamos a utilizar: "industria Hotelera de EEuu". Trabajamos con el dataset "Sitios"

import pandas as pd
import os
from datetime import datetime
import ast
import sys
import re
import reverse_geocoder as rg
import numpy as np


# Funciones

In [2]:

def get_urls(url):
    rutas = []
    for folder in os.walk(url):
        rutas.append(folder)
    df = pd.DataFrame(rutas, columns =['carpeta', 'sub_carpeta', 'archivos'])
    
    if df["archivos"][0] == []:
        df.drop(0,inplace=True)
    df = df.explode("archivos")
    df.reset_index(drop=True,inplace=True)
    df["carpeta"] = df["carpeta"].apply(lambda x: x.replace("\\","/"))
    df["ruta"] = df["carpeta"] + "/" + df["archivos"]
    return df


def get_reviews_url_info(serie_url, filtro, guardar=False):
    json_concat = pd.DataFrame()
    for i in serie_url:
        json_x = pd.read_json(i,lines=True)
        json_x["url_origen"] = i
        json_x = json_x[json_x["gmap_id"].isin(filtro["gmap_id"])]
        json_concat = pd.concat([json_concat,json_x])
    json_concat["etl_timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    json_concat["etl_timestamp"] = pd.to_datetime(json_concat["etl_timestamp"])
    json_concat.reset_index(drop=True,inplace=True)
    if guardar == True:
        json_concat.to_csv("d:/Henry/Proyecto Final/Dataset_generados/csv_{}.csv".format(datetime.now().strftime("%Y%m%d_%H%M%S")))
    return json_concat


def only_dict(d):
    '''
    Convert json string representation of dictionary to a python dict
    A = json_normalize(df['columnA'].apply(only_dict).tolist()).add_prefix('columnA.')
    '''
    return ast.literal_eval(str(d))

def list_of_dicts(ld):
    '''
    Create a mapping of the tuples formed after 
    converting json strings of list to a python list
    B = json_normalize(df['columnB'].apply(list_of_dicts).tolist()).add_prefix('columnB.pos.') 
    '''
    return dict([(list(d.values())[1], list(d.values())[0]) for d in ast.literal_eval(ld)])

def formatear_columnas(df_columnas,df_to_merge):
    '''Esta funcion sirve para darle formato a los nombres de las columnas dinamizadas y luego hacer un merge con otro dataframe por su mismo indice'''
    df_columnas.columns = df_columnas.columns.str.strip()
    df_columnas.columns = df_columnas.columns.str.replace(" ","_")
    df_columnas.columns = df_columnas.columns.str.replace("&","")
    df_columnas.columns = df_columnas.columns.str.replace("__","_")
    df_final = df_to_merge.merge(df_columnas,left_index=True, right_index=True,how="left")
    return df_final

def get_gmap_id(url_series):
    df = pd.DataFrame()
    for url in url_series:
        df_json = pd.read_json(url, lines=True)
        df_json = df_json[["name","gmap_id","category"]]
        df_categorias = df_json.explode("category")
        df_categorias["category"] = df_categorias["category"].str.lower()
        df_categorias = df_categorias[(df_categorias["category"].str.contains('hotel|resort|inn|motel|lodg',regex = True))== True] #agregar lodge ganamos 2k sitios mas
        df = pd.concat([df,df_categorias])
    return df

def reduccion_categoria(categoria):
    if "dinner" in categoria:
        return "Dinner"
    elif "lodg" in categoria:
        return "Lodge"
    elif "motel" in categoria:
        return "Motel"
    elif "resort" in categoria:
        return "Resort"
    elif "inn" in categoria:
        return "Inn"
    elif "hotel" in categoria:
        return "Hotel"
    else: return "Otros"

def list_to_lower(my_list):
    return [x.lower() for x in my_list]

def dinamizar_lista_a_columna(df,serie):
    lista_cat = df[serie].value_counts().index.to_list()
    flat_list = [item for sublist in lista_cat for item in sublist]
    for word in set(flat_list):
        if pd.isna(word):
            continue
        else: 
            df[word] = df[serie].dropna().apply(lambda x: 1 if word in x else 0)
            df[word].fillna(0,inplace=True)

def get_state(texto):
    if pd.isna(texto):
        return None
    else:
        pattern = re.compile(r'(\s{1}[A-Za-z]{2})(\s{1}\d{5})')
        text = re.search(pattern,texto)
        if bool(text):
            return text.group(1)

us_state_to_abbrev = {
'ALASKA':'AK',
'ARIZONA':'AZ',
'ARKANSAS':'AR',
'CALIFORNIA':'CA',
'COLORADO':'CO',
'CONNECTICUT':'CT',
'DELAWARE':'DE',
'FLORIDA':'FL',
'GEORGIA':'GA',
'HAWAII':'HI',
'IDAHO':'ID',
'ILLINOIS':'IL',
'INDIANA':'IN',
'IOWA':'IA',
'KANSAS':'KS',
'KENTUCKY':'KY',
'LOUISIANA':'LA',
'MAINE':'ME',
'MARYLAND':'MD',
'MASSACHUSETTS':'MA',
'MICHIGAN':'MI',
'MINNESOTA':'MN',
'MISSISSIPPI':'MS',
'MISSOURI':'MO',
'MONTANA':'MT',
'NEBRASKA':'NE',
'NEVADA':'NV',
'NEW HAMPSHIRE':'NH',
'NEW JERSEY':'NJ',
'NEW MEXICO':'NM',
'NEW YORK':'NY',
'NORTH CAROLINA':'NC',
'NORTH DAKOTA':'ND',
'OHIO':'OH',
'OKLAHOMA':'OK',
'OREGON':'OR',
'PENNSYLVANIA':'PA',
'RHODE ISLAND':'RI',
'SOUTH CAROLINA':'SC',
'SOUTH DAKOTA':'SD',
'TENNESSEE':'TN',
'TEXAS':'TX',
'UTAH':'UT',
'VERMONT':'VT',
'VIRGINIA':'VA',
'WASHINGTON':'WA',
'WEST VIRGINIA':'WV',
'WISCONSIN':'WI',
'WYOMING':'WY',
'ALABAMA':'AL',
'DISTRICT OF COLUMBIA':'DC'
}

abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

def get_descrip_state(texto):
    for i in us_state_to_abbrev.keys():
        if i in texto:
            return i
            exit

## Segmentacion de datos para industría Hotelera

In [3]:
# vamos a mapear todos los json, extraer todos los sitios con categoria hotel, motel, lodge, resort y los vamos acumular en una tabla para poder segmentar otros archivos
#este proceso toma 4 minutos
url_sitios = "d:/Henry/Proyecto Final/Data/Sitios"
rutas_sitios = get_urls(url_sitios)
df_gmap_id = get_gmap_id(rutas_sitios["ruta"])
df_gmap_id.drop_duplicates(subset="gmap_id",inplace=True)
df_gmap_id.reset_index(drop=True,inplace=True)
df_gmap_id["gmap_id"].to_csv("d:/Henry/Proyecto Final/Data/gmap_id.csv")

## Carga incremental

Codigo de carga para Sitios

In [4]:
#3 min para leer 11 jsons
#cargar lista de filtros de categorias
filtros = pd.read_csv("d:/Henry/Proyecto Final/Data/gmap_id.csv",index_col=0, names=["gmap_id"])
#cargar lista de rutas ya cargadas
rutas_sitios = pd.read_csv("d:/Henry/Proyecto Final/Data/rutas_sitios.csv")
#mapear carpetas para cargar
url_sitios = "d:/Henry/Proyecto Final/Data/Sitios"
df_url_sitios = get_urls(url_sitios)
#para realizar una carga de solo las cosas nuevas, vamos a realizar una limpieza entre las url_cargadas vs las mapeadas y borrar duplicados, de esa forma solo pasamos lo nuevo.
############ desactivar esta linea para realizar cargas iniciales #######################
df_url_sitios = df_url_sitios[~df_url_sitios["ruta"].isin(rutas_sitios["ruta"])]
#########################################################################################
#si no encuentra rutas nuevas, vamos a detener el script
if len(df_url_sitios) == 0:
    sys.exit()
#preparamos los datos para lo nuevo
sitios_hoteles = get_reviews_url_info(df_url_sitios["ruta"],filtros,True)
#vamos a actualizar la tabla de repetidos, en base las nuevas rutas ya cargadas
with open('d:/Henry/Proyecto Final/Data/rutas_sitios.csv', 'a', newline="") as f:
    df_url_sitios.to_csv(f, index=False, header=False)

from google.cloud import storage
path_to_private_key = 'd:/Henry/Proyecto Final/Carga incremental/genial-core-378003-6eb2af8349db.json'
client = storage.Client.from_service_account_json(json_credentials_path=path_to_private_key)

# The bucket on GCS in which to write the CSV file
bucket = client.bucket('raw_data_vault')
# The name assigned to the CSV file on GCS
blob = bucket.blob('sitios_hoteles_{}.csv'.format(datetime.now().strftime("%Y%m%d_%H%M%S")))
blob.upload_from_string(sitios_hoteles.to_csv(), 'text/csv')

Codigo de carga de Reviews

In [5]:
#21 min para revisar 51 estaods con 10json cada uno
#cargar lista de filtros de categorias
filtros = pd.read_csv("d:/Henry/Proyecto Final/Data/gmap_id.csv",index_col=0, names=["gmap_id"])
#cargar lista de rutas ya cargadas
rutas_reviews = pd.read_csv("d:/Henry/Proyecto Final/Data/rutas_reviews.csv")
#mapear carpetas para cargar
url_reviews = "d:/Henry/Proyecto Final/Data/Reviews"
df_url_reviews = get_urls(url_reviews)
#para realizar una carga de solo las cosas nuevas, vamos a realizar una limpieza entre las url_cargadas vs las mapeadas y borrar duplicados, de esa forma solo pasamos lo nuevo.
############ desactivar esta linea para realizar cargas iniciales #######################
#df_url_reviews = df_url_reviews[~df_url_reviews["ruta"].isin(rutas_reviews["ruta"])]
#########################################################################################
#si no encuentra rutas nuevas, vamos a detener el script
if len(df_url_reviews) == 0:
    sys.exit()
#preparamos los datos para lo nuevo
reviews_hoteles = get_reviews_url_info(df_url_reviews["ruta"],filtros,True)
#vamos a actualizar la tabla de repetidos, en base las nuevas rutas ya cargadas
with open('d:/Henry/Proyecto Final/Data/rutas_reviews.csv', 'a', newline="") as f:
    df_url_reviews.to_csv(f, index=False, header=False)

from google.cloud import storage
path_to_private_key = 'd:/Henry/Proyecto Final/Carga incremental/genial-core-378003-6eb2af8349db.json'
client = storage.Client.from_service_account_json(json_credentials_path=path_to_private_key)

# The bucket on GCS in which to write the CSV file
bucket = client.bucket('raw_data_vault')
# The name assigned to the CSV file on GCS
blob = bucket.blob('reviews_hoteles_{}.csv'.format(datetime.now().strftime("%Y%m%d_%H%M%S")))
blob.upload_from_string(reviews_hoteles.to_csv(), 'text/csv')

## Preparacion de datos

Lectura de datos

In [6]:
## Se leen los dos archivos que tienen el filtro realizado de "Industria Hotelera"

reviews_hotels = pd.read_csv("d:/Henry/Proyecto Final/Dataset_Consolidado/reviews_hoteles.csv",index_col=0)
sitios_hotels = pd.read_csv("d:/Henry/Proyecto Final/Dataset_Consolidado/sitios_hoteles.csv",index_col=0)

Prepracion sitios_hotel

In [7]:
#### tiempo ejecucion 3min ####
#pasamos a mayus la dir
sitios_hotels["address"] = sitios_hotels["address"].str.upper()
#corremos la funcion de regular expression que sirve para encontrar_AA_#####, que seria a abreviatura y el codigo postal.
sitios_hotels["state_xx"] = sitios_hotels["address"].apply(lambda x: get_state(x))
#pasamos el diccionario para saber a que estado se refiere el proyecto.
sitios_hotels["state_name"] = sitios_hotels["state_xx"].str.strip().map(abbrev_to_us_state)
#para aquellos que sí encontró, vamos a sacar la ciudad.
sitios_hotels["city_name"] = sitios_hotels.apply(lambda row: np.nan if pd.isnull(row.state_xx) else (row.address.split(",")[2].strip() if len(row.address.split(","))==4 else np.nan),axis=1)
#para aquellos que no encontramos, vamos a correr la funcion para hallar segun lat y long
##primero creamos la columnas de tuplas lat,long
sitios_hotels["coordenadas"] = list(zip(sitios_hotels["latitude"],sitios_hotels["longitude"]))
sitios_hotels["reverse_coor"] = sitios_hotels.apply(lambda row: rg.search(row.coordenadas) if pd.isnull(row.state_name) else np.nan, axis=1)
sitios_hotels["state_name"] = sitios_hotels.apply(lambda row: row.reverse_coor[0]["admin1"] if pd.notnull(row.reverse_coor) else row.state_name, axis=1)
sitios_hotels["state_name"] = sitios_hotels["state_name"].str.upper()
sitios_hotels["state_name"] = sitios_hotels["state_name"].str.replace("DISTRICT OF COLUMBIA","WASHINGTON, D.C.")
sitios_hotels["city_name"] = sitios_hotels.apply(lambda row: row.reverse_coor[0]["admin2"] if pd.notnull(row.reverse_coor) else row.city_name, axis=1)
sitios_hotels["city_name"] = sitios_hotels["city_name"].str.upper()
sitios_hotels['state_name'] = sitios_hotels.state_name.str.title()
sitios_hotels['city_name'] = sitios_hotels.city_name.str.title()

Loading formatted geocoded file...


AttributeError: 'float' object has no attribute 'title'

In [11]:
#abrir columna MISC
df_misc = sitios_hotels.dropna(subset="MISC")
df_misc_dinamizado = pd.json_normalize(df_misc["MISC"].apply(only_dict).tolist())
sitios_hotels = formatear_columnas(df_misc_dinamizado,sitios_hotels)

In [12]:
#clasificar los hoteles
sitios_hotels["category"] = sitios_hotels["category"].apply(only_dict).tolist()
sitios_hotels["category"] = sitios_hotels["category"].apply(list_to_lower)
sitios_hotels["cat_name"] = sitios_hotels.name.apply(lambda x: reduccion_categoria(x.lower()) if pd.notnull(x) else x)
sitios_hotels["cat_name"] = sitios_hotels.apply(lambda row: reduccion_categoria(str(row.category)) if row.cat_name == "Otros" else row.cat_name, axis=1)
sitios_hotels.drop(sitios_hotels[sitios_hotels["cat_name"]=="Dinner"].index,inplace=True)
sitios_hotels.reset_index(drop=True,inplace=True)

In [13]:
#abrir columnas creadas por MISC
for i in df_misc_dinamizado.columns:
    dinamizar_lista_a_columna(sitios_hotels,i)
sitios_hotels.drop(df_misc_dinamizado.columns,axis=1,inplace=True)

C:\Users\wongp\AppData\Local\Temp\ipykernel_17532\1610941285.py:91: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[word] = df[serie].dropna().apply(lambda x: 1 if word in x else 0)
C:\Users\wongp\AppData\Local\Temp\ipykernel_17532\1610941285.py:91: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[word] = df[serie].dropna().apply(lambda x: 1 if word in x else 0)
C:\Users\wongp\AppData\Local\Temp\ipykernel_17532\1610941285.py:91: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.in

In [14]:
sitios_num_col = len(sitios_hotels.columns)

In [15]:
#Agrupacion de columnas por servicios importantes

sitios_hotels["t.wheelchair"] = \
    sitios_hotels["Wheelchair accessible elevator"] + \
    sitios_hotels["Wheelchair accessible entrance"] + \
    sitios_hotels["Wheelchair accessible parking lot"] + \
    sitios_hotels["Wheelchair accessible restroom"] + \
    sitios_hotels["Wheelchair accessible seating"] + \
    sitios_hotels["Wheelchair-accessible car park"] + \
    sitios_hotels["Wheelchair-accessible entrance"] + \
    sitios_hotels["Wheelchair-accessible lift"] + \
    sitios_hotels["Wheelchair-accessible seating"] + \
    sitios_hotels["Wheelchair-accessible toilet"]

sitios_hotels["t.restaurant"] = \
    sitios_hotels["All you can eat"] + \
    sitios_hotels["Braille menu"] + \
    sitios_hotels["Catering"] + \
    sitios_hotels["Comfort food"] + \
    sitios_hotels["Dessert"] + \
    sitios_hotels["Dine-in"] + \
    sitios_hotels["Dinner"] + \
    sitios_hotels["Dinner reservations recommended"] + \
    sitios_hotels["Drive-through"] + \
    sitios_hotels["Fast service"] + \
    sitios_hotels["Food"] + \
    sitios_hotels["Great coffee"] + \
    sitios_hotels["Great dessert"] + \
    sitios_hotels["Great tea selection"] + \
    sitios_hotels["Halal food"] + \
    sitios_hotels["Happy hour food"] + \
    sitios_hotels["Happy-hour food"] + \
    sitios_hotels["Healthy options"] + \
    sitios_hotels["In-store pickup"] + \
    sitios_hotels["In-store shopping"] + \
    sitios_hotels["Kids' menu"] + \
    sitios_hotels["Late-night food"] + \
    sitios_hotels["Lunch"] + \
    sitios_hotels["Organic dishes"] + \
    sitios_hotels["Outside food allowed"] + \
    sitios_hotels["Quick bite"] + \
    sitios_hotels["Restaurant"] + \
    sitios_hotels["Small plates"] + \
    sitios_hotels["Solo dining"] + \
    sitios_hotels["Vegetarian options"] 

sitios_hotels["t.bar"] = \
    sitios_hotels["Alcohol"] + \
    sitios_hotels["Bar games"] + \
    sitios_hotels["Bar on site"] + \
    sitios_hotels["Bar onsite"] + \
    sitios_hotels["Beer"] + \
    sitios_hotels["Cocktails"] + \
    sitios_hotels["Food at bar"] + \
    sitios_hotels["Great bar food"] + \
    sitios_hotels["Great beer selection"] + \
    sitios_hotels["Great cocktails"] + \
    sitios_hotels["Great wine list"] + \
    sitios_hotels["Happy hour drinks"] + \
    sitios_hotels["Happy-hour drinks"] + \
    sitios_hotels["Hard liquor"] + \
    sitios_hotels["Salad bar"] + \
    sitios_hotels["Spirits"] + \
    sitios_hotels["Wine"]

sitios_hotels["t.child_care"] = \
    sitios_hotels["Child care"] + \
    sitios_hotels["Good for kids"] + \
    sitios_hotels["High chairs"]

sitios_hotels["t.familiar/group"] = \
    sitios_hotels["Family friendly"] + \
    sitios_hotels["Family-friendly"] + \
    sitios_hotels["Groups"]

sitios_hotels["t.all_payment_method"] = \
    sitios_hotels["Checks"] + \
    sitios_hotels["Cheques"] + \
    sitios_hotels["Credit cards"] + \
    sitios_hotels["Debit cards"] + \
    sitios_hotels["Membership required"] + \
    sitios_hotels["NFC mobile payments"]

sitios_hotels["t.lgtbq_friendly"] = \
    sitios_hotels["Gender-neutral restroom"] + \
    sitios_hotels["Gender-neutral toilets"] + \
    sitios_hotels["LGBTQ friendly"] + \
    sitios_hotels["LGBTQ-friendly"] + \
    sitios_hotels["Transgender safespace"]

sitios_hotels["t.outdoor_seating"] = \
    sitios_hotels["Outdoor seating"] + \
    sitios_hotels["Rooftop seating"] + \
    sitios_hotels["Seating"] + \
    sitios_hotels["Stadium seating"]

sitios_hotels["t.breakfast"] = \
    sitios_hotels["Breakfast"] + \
    sitios_hotels["Coffee"]

sitios_hotels["t.delivery"] = \
    sitios_hotels["Delivery"] + \
    sitios_hotels["Takeaway"] + \
    sitios_hotels["Takeout"]

sitios_hotels["t.live_performance"] = \
    sitios_hotels["Live music"] + \
    sitios_hotels["Live performances"]

sitios_hotels["t.covid"] = \
    sitios_hotels["Mask required"] + \
    sitios_hotels["No-contact delivery"] + \
    sitios_hotels["Staff get temperature checks"] + \
    sitios_hotels["Staff required to disinfect surfaces between visits"] + \
    sitios_hotels["Staff wear masks"] + \
    sitios_hotels["Temperature check required"]

sitios_hotels["t.fireplace"] = \
    sitios_hotels["Fireplace"]

#dropeamos todas  las columnas que no vamos a usar para nada
sitios_hotels.drop(sitios_hotels.iloc[:,23:sitios_num_col].columns,inplace=True,axis=1)
sitios_hotels.drop_duplicates(subset=['name','address','latitude','longitude'],inplace=True)
sitios_hotels.reset_index(inplace=True,drop=True)
sitios_hotels.rename(columns={"name":"name_hotel"},inplace=True)


C:\Users\wongp\AppData\Local\Temp\ipykernel_17532\2465753634.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sitios_hotels["t.wheelchair"] = \
C:\Users\wongp\AppData\Local\Temp\ipykernel_17532\2465753634.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sitios_hotels["t.restaurant"] = \
C:\Users\wongp\AppData\Local\Temp\ipykernel_17532\2465753634.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

In [16]:
sitios_hotels.to_csv('d:/Henry/Proyecto Final/Dataset_Consolidado/sitios_hotel_para_Eda.csv')

Preparacion reviews_hotels

In [17]:
# Se transforma la columna tiempo a fecha
reviews_hotels["time"]= pd.to_datetime(reviews_hotels["time"],unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S')
reviews_hotels["response"] = reviews_hotels["resp"].apply(lambda x: 0 if pd.isna(x) else 1 )
reviews_hotels["state_from_json_url"] = reviews_hotels["url_origen"].apply(lambda x: x.split("/")[5].split("-")[1])
reviews_hotels.drop_duplicates(subset=['name','time','text','rating','gmap_id','pics', 'resp'],inplace=True)
reviews_hotels.reset_index(inplace=True,drop=True)

In [18]:
reviews_hotels.to_csv('d:/Henry/Proyecto Final/Dataset_Consolidado/reviews_hotel_para_Eda.csv')